In [1]:
import tensorflow as tf

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def customOps(n):
    
    customOps.A = tf.placeholder(tf.float32, shape = [n, n])    
    
    # 1.Transpose the elements in the bottom-right triangle of A 
    
    A_transpose = tf.transpose(customOps.A)                   # transpose of A
    A_reverse = tf.reverse(customOps.A, [-1])                 # mirror image of A transpose
    A_transpose_reverse = tf.reverse(A_transpose, [-1])       # mirror image of A transpose
    upper =  tf.matrix_band_part(A_reverse, 0, -1)          
    lower = tf.matrix_band_part(A_transpose_reverse, -1, 0)
    final_upper = tf.reverse(upper, [-1])
    final_lower = tf.reverse(lower, [-1])
    diagonal = tf.matrix_band_part(A_reverse, 0, 0)  # diagonal matrix
    rev_diagonal = tf.reverse(diagonal, [-1])
    B = final_upper + final_lower - rev_diagonal
    

    # 2.Take the maximum value along the columns of A to get a vector m~ 
    
    m = tf.reduce_max(B, axis = 1)
    
    
    # 3.softmax matrix 
    
    B = tf.reshape(tf.tile(m,[n]),[n,n])
    rev1 = tf.reverse(B, [-1])
    mask_a = tf.matrix_band_part(rev1, 0, -1) # Upper triangular matrix of rev1
    rev2 = tf.reverse(mask_a1, [-1])
    B = tf.nn.softmax(B, axis = 0)
            
    
    # 4.Sum along the rows of B to obtain vector ~v 1 
    
    v1 = tf.reduce_sum(B, axis = 1)
    
    
    # Sum along the columns of B to get another vector ~v 2 
    
    v2 = tf.reduce_sum(B, axis = 0)
    
    
    # 5.Concatenate the two vectors and take a softmax of this vector: ~v = softmax(concat(~v 1 , ~v 2 ))
    
    v  = tf.nn.softmax(tf.concat([v1, v2], axis = 0), axis = 0)
    
    
    # 6.Get the index number in vector ~v with maximum value
    
    index = tf.argmax(v, axis = 0)
    
    
    # 7.If index number is greater than n/3 : 
    #      finalVal = ||v 1 − ~v 2 || 2  
    # else :
    #      finalval = ||v 1 + ~v 2 || 2
    
    finalVal = tf.cond(index > tf.cast((n/3),tf.int64), lambda : tf.square(v1-v2), lambda : tf.square(v1+v2))
    
    
    return finalVal

In [3]:
import numpy as np

if __name__ == '__main__':
    mat = np.asarray([[0, 1, 2],
                      [1, 0, 3],
                      [2, 5, 4]])
    n = mat.shape[0]

    finalVal = customOps(n)

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    outVal = sess.run(finalVal, feed_dict={customOps.A: mat})
    print(outVal)
    sess.close()

NameError: name 'mask_a1' is not defined

In [ ]:
with tf.Session() as sess:
    mat = np.asarray([[1, 2, 3, 4],
                      [5, 6, 7, 8],
                      [9, 10, 11, 12],
                      [13, 14, 15, 16]])
    
#     mask_a = tf.matrix_band_part(mat, 0, -1) # Upper triangular matrix of 0s and 1s
#     rev = tf.reverse(mask_a, [-1])
    rev1 = tf.reverse(mat, [-1])
    mask_a1 = tf.matrix_band_part(rev1, 0, -1) # Upper triangular matrix of 0s and 1s
    rev2 = tf.reverse(mask_a1, [-1])
#     mask_b = tf.matrix_band_part(B, 0, 0)  # Diagonal matrix of 0s and 1s
#     mask = tf.cast(mask_a - mask_b, dtype=tf.bool) # Make a bool mask

#     upper_triangular_flat = tf.boolean_mask(B, mask)
    
#     print(sess.run(mat))
    print(sess.run(rev2))    

In [ ]:
# m1 = [1,2,3,4]
# a = tf.stack([m1 for i in range(10)] )
# rev1 = tf.reverse(mat, [-1])
# mask_a1 = tf.matrix_band_part(rev1, 0, -1) # Upper triangular matrix of 0s and 1s
# rev2 = tf.reverse(mask_a1, [-1])
# # B = tf.nn.softmax(rev2, axis = 0)
m = tf.reduce_max(mat, axis = 1)


with tf.Session() as sess:
#     print(sess.run(rev2))
    print(sess.run(m))    

In [ ]:
customOps.A = mat    
    
# 1.Transpose the elements in the bottom-right triangle of A 
B = tf.transpose(customOps.A)
rev_upper = tf.reverse(customOps.A, [-1])
rev_lower = tf.reverse(B, [-1])
upper =  tf.matrix_band_part(rev_upper, 0, -1)
lower = tf.matrix_band_part(rev_lower, -1, 0)
final_upper = tf.reverse(upper, [-1])
final_lower = tf.reverse(lower, [-1])
diagonal = tf.matrix_band_part(rev_lower, 0, 0)
rev_diagonal = tf.reverse(diagonal, [-1])
val = final_upper + final_lower - rev_diagonal

n = 3
m = tf.reduce_max(B, axis = 1)
b = tf.reshape(tf.tile(m,[n]),[n,n])
# b = tf.tile(m,[n])

with tf.Session() as sess:
#     print(sess.run(customOps.A))
#     print(sess.run(upper))
#     print(customOps.A)
#     print(sess.run(rev_upper))
#     print(sess.run(B))
#     print(sess.run(rev_lower))
#     print(sess.run(final_upper))
#     print(sess.run(final_lower))   
#     print(sess.run(diagonal))
#     print(sess.run(m))
    
    print(sess.run(b))